In [47]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pywt
from math import log10, sqrt
import os
from skimage import exposure

* Embedding

In [48]:
import random
def hcf(a,b):
    if(b%a==0):
        return a
    else:
        return hcf(b%a,a)
def randomseq(size):
    random.seed(size)
    
    num=[x for x in range(size)]
    h=len(num)
    for m in range(h):
        if m>2:
            r=hcf(m,h)
            if r==1:
                break 
    i=random.randint(0,size-1)    
    x=i
    res=[]
    while True:
        i=(i+m)%h
        if i==x:
            break
        res.append(i)
        
    res.append(x)
    return res

In [49]:
def embedding(img,wtr_img):
    wrow,wcol=wtr_img.shape
    LL,(HL,LH,HH)=pywt.dwt2(img,wavelet='haar')
    l2x2=LL.reshape(-1,2,2)
    sstore=[]
    ustore=[]
    vstore=[]
    for i in l2x2:
        u,s,v=np.linalg.svd(i)
        ustore.append(u)
        vstore.append(v)
        sstore.append(s)
    ss=np.array(sstore)
    ss_backup=ss.copy()
    h2x2=HH.reshape(-1,2,2) #Store the ss_backup into HH
    uu=[]
    sss=ss_backup.ravel()
    maxs=max(sss)
    stemp=[]
    vv=[]
    newHH=[]
    for i in h2x2:
        u,s,v=np.linalg.svd(i)
        uu.append(u)
        vv.append(v)
        stemp.append(s)
    stemp=np.array(stemp)
    stemp=ss_backup
    for i in range(len(stemp)):
        u=uu[i]
        v=vv[i]
        s=stemp[i]/maxs
        temp=u@np.diag(s)@v
        newHH.append(temp)
    newHH=np.array(newHH)
    newHH=newHH.ravel()
    newHH=newHH.reshape(256,256)
    HH=newHH
    wtr_list=wtr_img.ravel()
    j=0
    seqnum=randomseq(wrow*wcol)
    for i in seqnum:
        ss[i][0]+=wtr_list[j]/256
        j+=1    
    newll=[]
    for i in range(len(ss)):
        u=ustore[i]
        v=vstore[i]
        s=ss[i]
        temp=u@np.diag(s)@v
        newll.append(temp)
    newll=np.array(newll)
    newll=newll.ravel()
    row,col=LL.shape
    newll=newll.reshape(row,col)
    watermarked_img=pywt.idwt2((newll,(HL,LH,HH)),wavelet='haar')
    return watermarked_img,maxs

* Watermark Extraction

In [50]:
def recover(watermarked_img,maxs,wrow,wcol):
    dLL,(dHL,dLH,dHH)=pywt.dwt2(watermarked_img,wavelet='haar')
    dl2x2=dLL.reshape(-1,2,2)
    dh2x2=dHH.reshape(-1,2,2)
    sstore2=[]
    ustore=[]
    vstore=[]
    uu2=[]
    vv2=[]
    ss2=[]
    for i in dh2x2:
        u,s,v=np.linalg.svd(i)
        # s=s.round()
        uu2.append(u)
        vv2.append(v)
        ss2.append(s*maxs)
    ss2=np.array(ss2)
    for i in dl2x2:
        u,s,v=np.linalg.svd(i)
        # s=s.round()
        ustore.append(u)
        vstore.append(v)
        sstore2.append(s)
    sstore2=np.array(sstore2)
    rec_wtr=[]
    seqnum=randomseq(wrow*wcol)
    for i in seqnum:
        temp=(sstore2[i][0]-ss2[i][0])*256
        rec_wtr.append(temp)
    rec_wtr=np.array(rec_wtr)
    final_img=rec_wtr.reshape(wrow,wcol)
    temp_im=np.round(final_img)
    for i in range(len(temp_im)):
        for j in range(len(temp_im[0])):
            temp_im[i][j]=int(temp_im[i][j])
    return temp_im

* Load Images

In [51]:
def img_read(x):
    img=cv2.imread(x,0)
    return img
def watermark_read(x):
    wtr_img=cv2.imread(x,0)
    wtr_img=cv2.resize(wtr_img,(32,32))
    return wtr_img

* Testing

* Results

In [52]:
from sklearn.metrics import mean_squared_error
from skimage.metrics import structural_similarity
from PIL import Image

def ssim(name,name1):
    return structural_similarity(name,name1)
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    try:
        max_pixel = 255.0
        psnr = 20 * log10(max_pixel / sqrt(mse))
        return psnr
    except:
        return -9999  #When MSE is Zero, i.e, no noise is present in the image
def norm_data(data):
    mean_data=np.mean(data)
    std_data=np.std(data,ddof=1)
    return (data-mean_data)/(std_data)
def ncc(data0,data1):
    c=(1.0/(data0.size-1))*np.sum(norm_data(data0)*norm_data(data1))
    return c

In [53]:
# print("PSNR of Cover Image : ",PSNR(img,watermarked_img))
# print("PSNR of watermark : ",PSNR(wtr_img,extracted_img))
# print("MSE of Original Image : ",mean_squared_error(img,watermarked_img))
# print("MSE of Watermark Image : ",mean_squared_error(wtr_img,extracted_img))

* Various Attacks

In [54]:
def salt_pepper(img,p):
    new=img.copy()
    x,y=img.shape
    pixels=int(x*y*p)
    sp=[]
    for i in range(pixels):
        sp.append((random.randint(0,y-1),random.randint(0,x-1),random.choice([0,255])))
    for i,j,v in sp:
        new[i,j]=v
    return new

In [55]:
def gauss_noise(img,mean,variance):
    s=variance**2
    x,y=img.shape
    r=np.random.normal(mean,s,(y,x))
    return img+r

In [56]:
def crop(img,d):
    new=img.copy()
    x,y=img.shape
    x1=(x-d)//2
    y1=(y-d)//2
    new[y1:y1+d,x1:x1+d]=255
    return new

In [57]:
def jpeg_conversion(name,img,qf):
    new=Image.fromarray(img)
    new.save(name,quality=qf)

* Applying Attacks

In [58]:
def allatk(im,name):
    images=[]
    images.append(crop(im,50))
    images.append(gauss_noise(im,1,2))
    # images.append(cv2.imwrite(f'./Output/METHOD1/jpeg/{name}.jpeg',im))
    images.append(salt_pepper(im,0.001))
    images.append(cv2.equalizeHist(im))
    images.append(cv2.rotate(im,cv2.ROTATE_90_CLOCKWISE))
    return images

In [59]:
# t1=gauss_noise(watermarked_img,0,0.01)
# plt.imshow(t1,cmap='gray')

In [60]:
# im2=extraction(t1,stego_key,wrow,wcol)
# plt.imshow(im2,cmap='gray')
# PSNR(wtr_img,im2)

In [61]:
def rotateMatrix(matrix):
    mat=matrix.copy()
    if not len(mat):
        return
     
    """
        top : starting row index
        bottom : ending row index
        left : starting column index
        right : ending column index
    """
 
    top = 0
    bottom = len(mat)-1
 
    left = 0
    right = len(mat[0])-1
 
    while left < right and top < bottom:
 
        # Store the first element of next row,
        # this element will replace first element of
        # current row
        prev = mat[top+1][left]
 
        # Move elements of top row one step right
        for i in range(left, right+1):
            curr = mat[top][i]
            mat[top][i] = prev
            prev = curr
 
        top += 1
 
        # Move elements of rightmost column one step downwards
        for i in range(top, bottom+1):
            curr = mat[i][right]
            mat[i][right] = prev
            prev = curr
 
        right -= 1
 
        # Move elements of bottom row one step left
        for i in range(right, left-1, -1):
            curr = mat[bottom][i]
            mat[bottom][i] = prev
            prev = curr
 
        bottom -= 1
 
        # Move elements of leftmost column one step upwards
        for i in range(bottom, top-1, -1):
            curr = mat[i][left]
            mat[i][left] = prev
            prev = curr
 
        left += 1
 
    return mat

In [62]:
def medianfilter(img):
    im=img.copy()
    rows=im.shape[0]
    cols=im.shape[1]
    for i in range(1,rows-1):
        for j in range(1,cols-1):
            temp_array=[im[i][j],im[i-1][j],im[i+1][j],im[i][j-1],im[i][j+1],im[i-1][j-1],im[i-1][j+1],im[i+1][j-1],im[i+1][j+1]]
            temp_array.sort()
            # print(temp_array)
            im[i][j]=temp_array[4]
            temp_array=None
    return im

In [72]:
images=os.listdir('../datasets/BOSSbase_1.01_DATASET/BOSSbase_1.01')
watermarks=os.listdir('./CoverImages')
imgpath='../datasets/BOSSbase_1.01_DATASET/BOSSbase_1.01'
wtrpath='./CoverImages'
ssim_list=[]
psnr_list=[]
imagelist=[]
watermarklist=[]
ncc_list=[]
c=0
test_wtr=[watermarks[0],watermarks[1],watermarks[7],watermarks[5],watermarks[6]]

# i=watermarks[0]
for i in test_wtr:
    count=0
    wtr_img=watermark_read(os.path.join(wtrpath,i))
    name_folder=i.split('.')
    name_folder=name_folder[0]
    path=f'D:/CS/Sem_6/Project/Practice/Output/METHOD1_32/salt&peeper/{name_folder}'
    os.mkdir(path)
    for j in images:
        img=img_read(os.path.join(imgpath,j))
        watermarked_img,stego_key=embedding(img,wtr_img)
        wrow,wcol=wtr_img.shape
        name="wtr_"+i+'_host_'+j
        cv2.imwrite(os.path.join(path,f'{name}_noAtk.jpg'),watermarked_img)
        # stego=cv2.imread(os.path.join(path,f'{name}_noAtk.jpg'),0)


        # atk=np.round(watermarked_img)
        # # for i in range(atk.shape[0]):
        # #     for j in range(atk.shape[1]):
        # #         atk[i][j]=int(atk[i][j])
        # atk=atk.astype('uint8')
        # atk=cv2.equalizeHist(atk)


        # atk=crop(watermarked_img,20)

        ###Rotation
        # import imutils
        # atk=imutils.rotate(watermarked_img,45)
        # generated_with_atk=recover(imutils.rotate(atk,-45),stego_key,wrow,wcol)
        # generated_img=recover(watermarked_img,stego_key,wrow,wcol)


        # atk=medianfilter(watermarked_img)
        atk=salt_pepper(watermarked_img,0.001)
        generated_with_atk=recover(atk,stego_key,wrow,wcol)
        generated_img=recover(watermarked_img,stego_key,wrow,wcol)
        

        # path=f'D:/CS/Sem 6/Project/Practice/Output/BOSBaseUsingUMatrixMethod/{name_folder}'
        cv2.imwrite(os.path.join(path,f'{name}_withAtk.jpg'),atk)
        cv2.imwrite(os.path.join(path,f'{name}_watermark.jpg'),generated_img)
        cv2.imwrite(os.path.join(path,f'{name}_watermarkOriginal.jpg'),wtr_img)
        cv2.imwrite(os.path.join(path,f'{name}_watermarkWithAtk.jpg'),generated_with_atk)
        # ssim_score=ssim(wtr_img,generated_with_atk)
        psnr_score=PSNR(img,watermarked_img)
        psnr_list.append(psnr_score)
        # ssim_list.append(ssim_score)
        temp1=cv2.imread(os.path.join(path,f'{name}_watermarkOriginal.jpg'))
        temp2=cv2.imread(os.path.join(path,f'{name}_watermarkWithAtk.jpg'))
        imagelist.append(j)
        watermarklist.append(i)
        nccscore=ncc(temp1,temp2)
        ncc_list.append(nccscore)
        count+=1
        if count>=20:
            break
    c+=1


In [73]:
print(ncc_list)

[0.9398859427407024, 0.912028269208311, 0.912028269208311, 0.912028269208311, 0.9200967956305864, 0.912028269208311, 0.912028269208311, 0.912028269208311, 0.9291646490582856, 0.9221639046703324, 0.912028269208311, 0.912028269208311, 0.912028269208311, 0.912028269208311, 0.912028269208311, 0.912028269208311, 0.9201500049735097, 0.912028269208311, 0.912028269208311, 0.912028269208311, 0.9197521186731903, 0.9043713678081966, 0.9043713678081966, 0.9043713678081966, 0.9053985636652039, 0.9043713678081966, 0.9043713678081966, 0.9043713678081966, 0.9098042430863859, 0.9134247925354159, 0.9043713678081966, 0.9043713678081966, 0.9043713678081966, 0.9043713678081966, 0.9043713678081966, 0.9043713678081966, 0.9043713678081966, 0.9043713678081966, 0.9043713678081966, 0.9043713678081966, 0.9413144167258145, 0.9308093691062631, 0.9308093691062631, 0.9308093691062631, 0.9399436781143793, 0.9308093691062631, 0.9308093691062631, 0.9308093691062631, 0.9287059168822416, 0.9271586818656609, 0.930809369106

In [74]:
import pandas as pd
col=['Image','Watermark','NCC','PSNR']
df=pd.DataFrame(columns=col)
df['Image']=imagelist
df['Watermark']=watermarklist
# df['SSIM']=ssim_list
df['NCC']=ncc_list
df['PSNR']=psnr_list
df.to_csv('./Output/METHOD1_32/32old_salt&peeper.csv')

In [66]:
# t1=cv2.imread('./Output/METHOD1/airplane/wtr_airplane.tiff_host_1.pgm_watermark.jpg',0)
# t2=cv2.imread('./Output/METHOD1/airplane/wtr_airplane.tiff_host_1.pgm_watermarkWithAtk.jpg',0)
# print(ncc(t1,t2))

In [67]:
# print(ncc_list)

* Test

In [68]:
# mat=[[1,2,3,4],[12,23,45,10],[3,60,34,5],[2,5,6,14]]
# print(mat)
# wtr_img=np.array([10])
# LL,(HL,LH,HH)=pywt.dwt2(mat,wavelet='haar')
# print("LL")
# print(LL)
# l2x2=LL.reshape(-1,2,2)
# sstore=[]
# ustore=[]
# vstore=[]
# for i in l2x2:
#     u,s,v=np.linalg.svd(i)
#     ustore.append(u)
#     vstore.append(v)
#     sstore.append(s)
# ss=np.array(sstore)
# print("U")
# print(ustore)
# print("S Matrix")
# print(ss)
# print("v")
# print(vstore)
# ss_backup=ss.copy()
# h2x2=HH.reshape(-1,2,2) #Store the ss_backup into HH
# uu=[]
# sss=ss_backup.ravel()
# maxs=max(sss)
# stemp=[]
# vv=[]
# newHH=[]
# for i in h2x2:
#     u,s,v=np.linalg.svd(i)
#     uu.append(u)
#     vv.append(v)
#     stemp.append(s)
# stemp=np.array(stemp)
# stemp=ss_backup
# for i in range(len(stemp)):
#     u=uu[i]
#     v=vv[i]
#     s=stemp[i]/maxs
#     temp=u@np.diag(s)@v
#     newHH.append(temp)
# newHH=np.array(newHH)
# newHH=newHH.ravel()
# newHH=newHH.reshape(2,2)
# # HH=newHH
# wtr_list=wtr_img.ravel()
# # print(wtr_list[3])
# j=0
# seqnum=randomseq(1)
# for i in seqnum:
#     ss[i][0]+=wtr_list[j]/60
#     j+=1    
# newll=[]
# print("S matrix after embedding")
# print(ss)
# for i in range(len(ss)):
#     u=ustore[i]
#     v=vstore[i]
#     s=ss[i]
#     temp=u@np.diag(s)@v
#     newll.append(temp)
# newll=np.array(newll)
# newll=newll.ravel()
# row,col=LL.shape
# newll=newll.reshape(row,col)
# print("LL after embedding")
# print(newll)
# watermarked_img=pywt.idwt2((newll,(HL,LH,HH)),wavelet='haar')
# print("Watermarked Data after embedding")
# print(watermarked_img)

In [69]:
# dLL,(dHL,dLH,dHH)=pywt.dwt2(watermarked_img,wavelet='haar')
# dl2x2=dLL.reshape(-1,2,2)
# dh2x2=dHH.reshape(-1,2,2)
# sstore2=[]
# ustore=[]
# vstore=[]
# uu2=[]
# vv2=[]
# ss2=[]
# # for i in dh2x2:
# #     u,s,v=np.linalg.svd(i)
# #     # s=s.round()
# #     uu2.append(u)
# #     vv2.append(v)
# #     ss2.append(s*maxs)
# # ss2=np.array(ss2)
# for i in dl2x2:
#     u,s,v=np.linalg.svd(i)
#     # s=s.round()
#     ustore.append(u)
#     vstore.append(v)
#     sstore2.append(s)
# sstore2=np.array(sstore2)
# rec_wtr=[]
# seqnum=randomseq(1)
# for i in seqnum:
#     temp=(sstore2[i][0]-stemp[i][0])*60
#     rec_wtr.append(temp)
# rec_wtr=np.array(rec_wtr)
# # print(rec_wtr)
# final_img=rec_wtr.reshape(1)
# temp_im=np.round(final_img)
# temp_im[0]=int(temp_im[0])
# print("Recovered Image")
# print(temp_im)

In [70]:
from tifffile import imsave,imread
im1=cv2.imread('./CoverImages/lena.tiff',0)
im2,skey=embedding(im1,wtr_img)
# cv2.imwrite('./tempim.png',im2)
# im2=cv2.imread('./tempim.png',0)
# LL,(HL,LH,HH)=pywt.dwt2(im2,wavelet='haar')

# LL=np.round(LL)
# # HL=np.round(HL)
# # LH=np.round(LH)
# # HH=np.round(HH)
# im2=pywt.idwt2((LL,(HL,LH,HH)),wavelet='haar')

# im2=im2.astype(int)
print(PSNR(im2,im1))
print(im2.dtype)
# im2=gauss_noise(im2,0.1,0.01)
# imsave('result.tif',im2)
# im2=imread('result.tif')
# plt.imshow(im2,cmap='gray')
# newim=recover(im2,skey,128,128)
# print(ncc(newim,wtr_img))
# plt.imshow(newim,cmap='gray')

42.5015044487379
float64
